# Data wrangling for *Mtb*-Tn-Mat project

* reads in SI tables / files and compiles them into Pandas dataframe 
* version 2.0: goal is to have 3 matrices:
    1. binary essentiality matrix

## Imports modules:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from statsmodels.stats.multitest import multipletests
from Tn_data_wrangling import *

%load_ext autoreload
%autoreload 2

pd.options.mode.chained_assignment = None  # default='warn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## function definitions:

*all function definitions have been moved to Tn_data_wrangling.py*

#### Read in full set of Rv_IDs and gene_names from tuberculist annotation: 

In [2]:
root_dir = '../..'
data_path = os.path.join(root_dir, 'data')

In [3]:
file_WG = 'annotations/H37Rv_tuberculist_functional_categories.xlsx'
df_WG = pd.read_excel(os.path.join(data_path, file_WG))
df_WG = df_WG[['Rv_ID', 'gene_name']]
df_WG.head(3)

,Rv_ID,gene_name
0,Rv0001,dnaA
1,Rv0002,dnaN
2,Rv0003,recF


## The "simple" cases:

These are the SI files and tables (19 columns) that are in a 'simple' standard format":

i.e they list the set of genes that are called (conditionally) essential in the Tn screen

In [27]:
# read in path/file_name to column name mapping for 'simple' cases:
col_name_file = os.path.join( data_path, 'files_and_columns.csv') 
df_col_info_simple = pd.read_csv(col_name_file)
df_col_info_simple.head(2)

,file,col_name,q_val_col_name,ratio_col_name,is_ratio_log2FC
0,2003_Sassetti_Boyd_Rubin/table_1.xls,2003A_Sassetti,p_val,ratio,FALSE
1,2003_Sassetti_Rubin/table_3.xls,2003B_Sassetti,Week_8_P_value,Week_8_Ratio,FALSE


#### Building binary matrix for simple cases: 

In [28]:
tn_data_path = os.path.join(root_dir, 'data/Tn_datasets')

In [29]:
counter = 0

for index, row in df_col_info_simple.iterrows():
    
    file_in = os.path.join(tn_data_path, row['file'])
    col_name = row['col_name']
    
    # build binary dataframe from single spreadsheet
    df_tn = build_single_column_df(file_in, col_name)
    df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
    
    # merge with rest of data-files / columns. 
    if counter == 0:
        df_tn_ALL = df_tn_WG.copy()
    else:
        df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
    
    counter +=1

In [30]:
df_tn_ALL.shape

(3990, 21)

## Special cases: 

These are SI datafiles and tables that are in different / unique formats, so it's more annoying to put them together into a single format. 

Load spreasheet with file name-to-column mappings

In [31]:
col_name_file_special = os.path.join( data_path, 'files_and_columns_set2.csv') 
df_col_info_special = pd.read_csv(col_name_file_special)
df_col_info_special = df_col_info_special[df_col_info_special.columns[:5]]
df_col_info_special.head(5)

,file,col_name,q_val_col_name,ratio_col_name,is_ratio_log2FC
0,2012_Zhang_Rubin/table_4_sheet_600bp.xlsx,2012_Zhang,p-val avg,NONE,NONE
1,2015_Kieser_Rubin/table_1A_edited.xlsx,2015_Kieser_GI_1,P-value,Fold Change,FALSE
2,2015_Kieser_Rubin/table_1B_edited.xlsx,2015_Kieser_GI_2,P-value,Fold Change,FALSE
3,2015_Kieser_Rubin/table_1C_edited.xlsx,2015_Kieser_GI_3,P-value,Fold Change,FALSE
4,2015_Mendum_Stewart/table_1A.xlsx,2015_Mendum,p_val_7days,NONE,NONE


### DeJesus and Iorger (2013)

essentiality matrix: 

In [32]:
###########
file = '2013_DeJesus_Iorger/table_1.xls'
col_name = '2013_DeJesus'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn[col_name] = df_tn.shape[0]*[0]
df_tn.loc[df_tn.Call == 'E', col_name] = 1
###############
### [OPTIONAL] Account for uncertains! 
############
df_tn = df_tn[['Rv_ID', col_name]]
###########
# merge with whole genome
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
# merge with all spreadsheets: 
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 22)

### Zhang and Rubin (2012)

essentiality matrix:

In [33]:
###########
# # (2012) Zhang, Rubin
# Description: 
# * Grown in 7H10 media (as in Sassetti et al., 2001)

file = '2012_Zhang_Rubin/table_4.xlsx'
col_name = '2012_Zhang'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn[col_name] = df_tn.shape[0]*[0]
df_tn.loc[df_tn.Call == 'E', col_name] = 1
df_tn.loc[df_tn.Call == 'D', col_name] = 1
df_tn = df_tn[['Rv_ID', col_name]]
# merge with whole genome
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
# merge with all spreadsheets: 
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 23)

### Kieser Rubin (2015 data) - tables A, B, C

In [34]:
file_list = ['2015_Kieser_Rubin/table_1A.xlsx', '2015_Kieser_Rubin/table_1B.xlsx', '2015_Kieser_Rubin/table_1C.xlsx']
col_name_list = ['2015_Kieser_GI_1', '2015_Kieser_GI_2', '2015_Kieser_GI_3']

essentiality matrix:

In [35]:
p_val_thresh = 0.05 # Talk to Michael and Anisha about these parameter values. 
fold_change_thresh = 2 # Talk to Michael and Anisha about these parameter value. 

for i in range(len(file_list)):
    file = file_list[i]
    col_name = col_name_list[i]

    df_tn = pd.read_excel(os.path.join(tn_data_path, file))
    df_tn = df_tn[~df_tn.Rv_ID.str.contains('IG')]
    rv_id_list = [rv_id.split('_')[-1].strip('\'') for rv_id in df_tn['Rv_ID'].values]
    df_tn['Rv_ID'] = rv_id_list

    df_tn[col_name] = df_tn.shape[0]*[0]
    df_tn.loc[(df_tn['P-value'] < p_val_thresh) & (df_tn['Fold Change'] > fold_change_thresh), col_name] = 1

    # merge with whole genome: 
    df_tn = df_tn[['Rv_ID', col_name]]
    df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
    # merge with all spreadsheets: 
    df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])


In [36]:
df_tn_ALL.shape

(3990, 26)

### Mendum and Stewart (2015) data

In [37]:
###########
# # (2015) Mendum, Stewart
# Description: 
# * virulence profile in dendritic cells
file = '2015_Mendum_Stewart/table_1A.xlsx'
col_name = '2015_Mendum'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn[col_name] = df_tn.shape[0]*[0]

# ## Mult. hypothesis testing correction (Benjamini/Hochberg)
pvals = df_tn.p_val_7days.values
ind_not_nan = [i for i in range(len(pvals)) if ~np.isnan(pvals[i])]
ind_nan = [i for i in range(len(pvals)) if np.isnan(pvals[i])]
pvals_not_nan = pvals[ind_not_nan]

reject, pvals_correct, alphacSidak, alphacBonf = multipletests(pvals_not_nan, method = 'fdr_bh')
pvals_new = np.zeros(len(pvals))

pvals_new[ind_not_nan] = pvals_correct
pvals_new[ind_nan] = np.nan
df_tn['p_val_7days'] = pvals_new


# keep p-adjusted < 0.05
df_tn.loc[df_tn.p_val_7days < (0.05), col_name] = 1
df_tn = df_tn[['Rv_ID', col_name]]
###########
# merge with whole genome
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
# merge with all spreadsheets: 
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 27)

### Nambi and Sassetti (2016) data:

In [38]:
###########
# # (2016) Nambi, Sassetti
# Description: 
# * genetic interaction with ctpC::hyg background
file = '2016_Nambi_Sassetti/table_1.xlsx'
col_name = '2016_Nambi'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn[col_name] = df_tn.shape[0]*[0]
df_tn.loc[df_tn.Qval < 0.05, col_name] = 1
df_tn = df_tn[['Rv_ID', col_name]]
###########
# merge with whole genome
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
# merge with all spreadsheets: 
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 28)

### Xu and Ehrt (2017) data:

In [39]:
file_list = ['2017_Xu_Ehrt/table_3A.xlsx',
            '2017_Xu_Ehrt/table_3B.xlsx',
            '2017_Xu_Ehrt/table_3C.xlsx',
            '2017_Xu_Ehrt/table_3D.xlsx',
            '2017_Xu_Ehrt/table_3E.xlsx']

col_name_list = ['2017_Xu_1A', '2017_Xu_1B', '2017_Xu_1C', '2017_Xu_1D', '2017_Xu_1E']

In [40]:
for i in range(len(file_list)):
    file = file_list[i]
    col_name = col_name_list[i]
    df_tn = pd.read_excel(os.path.join(tn_data_path, file))
    df_tn[col_name] = df_tn.shape[0]*[0]
    df_tn.loc[ (df_tn['qval'] < 0.05) , col_name] = 1
    df_tn = df_tn[['Rv_ID', col_name]]

    df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
    if i == 0:
        df_ALL_Xu = df_tn_WG.copy()
    else:
        df_ALL_Xu = df_ALL_Xu.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])

In [41]:
df_tn_ALL = df_tn_ALL.merge(df_ALL_Xu, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 33)

### Carey, Fortune (2018)

In [42]:
file_list = ['2018_Carey_Fortune/table_1A.xlsx',
            '2018_Carey_Fortune/table_1B.xlsx',
            '2018_Carey_Fortune/table_1C.xlsx',
            '2018_Carey_Fortune/table_1D.xlsx',
            '2018_Carey_Fortune/table_1E.xlsx',
            '2018_Carey_Fortune/table_1F.xlsx',
            '2018_Carey_Fortune/table_1G.xlsx',
            '2018_Carey_Fortune/table_1H.xlsx',]

col_name_list = ['2018_Carey_1A', '2018_Carey_1B', '2018_Carey_1C', '2018_Carey_1D',
                '2018_Carey_1E', '2018_Carey_1F', '2018_Carey_1G', '2018_Carey_1H']

In [43]:
for i in range(len(file_list)):
    file = file_list[i]
    col_name = col_name_list[i]
    df_tn = pd.read_excel(os.path.join(tn_data_path, file))
    df_tn[col_name] = df_tn.shape[0]*[0]
    df_tn.loc[ (df_tn['p-adj'] < 0.05) & (df_tn['log2 FC'].abs() > 1), col_name] = 1
    df_tn = df_tn[['Rv_ID', col_name]]

    df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
    if i == 0:
        df_ALL_Carey = df_tn_WG.copy()
    else:
        df_ALL_Carey = df_ALL_Carey.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])

In [44]:
df_tn_ALL = df_tn_ALL.merge(df_ALL_Carey, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 41)

### deJesus, Iorger (2017A) data

In [45]:
###########
# # (2017) DeJesus, Iorger
# Description: 
# 
file = '2017A_DeJesus_Iorger/table_1.xlsx'
col_name = '2017A_DeJesus'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn[col_name] = df_tn.shape[0]*[0]
df_tn.loc[ (df_tn['Final Call'] == 'ES') | (df_tn['Final Call'] == 'ESD')
          | (df_tn['Final Call'] == 'GD'), col_name] = 1
df_tn = df_tn[['Rv_ID', col_name]]
###########
# merge with whole genome
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
# merge with all spreadsheets: 
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 42)

# DeJesus, Iorger (2017B)

In [46]:
file_list = ['2017B_DeJesus_Iorger/table_1A.xlsx',
            '2017B_DeJesus_Iorger/table_1B.xlsx',
            '2017B_DeJesus_Iorger/table_1C.xlsx']

col_name_list = ['2017B_DeJesus_1A', '2017B_DeJesus_1B', '2017B_DeJesus_1C']

In [47]:
# # (2017) DeJesus, Iorger
# Description: 
# 
for i in range(len(file_list)):

    file = file_list[i]
    col_name = col_name_list[i]
    df_tn = pd.read_excel(os.path.join(tn_data_path, file))
    df_tn[col_name] = df_tn.shape[0]*[0]
    df_tn.loc[ (df_tn['Type of Interaction'] != 'No-Interaction'), col_name] = 1
    df_tn = df_tn[['Rv_ID', col_name]]
    ###########
    # merge with whole genome
    df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn)
    if i == 0:
        df_ALL_DJ = df_tn_WG.copy()
    else:
        df_ALL_DJ = df_ALL_DJ.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])

In [48]:
df_tn_ALL = df_tn_ALL.merge(df_ALL_DJ, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 45)

## FLUTE knockout interactions:

In [49]:
file = 'FLUTE_KO_TnSeq/table_1.xlsx'
df_tn = pd.read_excel(os.path.join(tn_data_path, file))
df_tn_WG = df_WG.merge(df_tn, how = 'left', on = 'Rv_ID')
df_tn_ALL = df_tn_ALL.merge(df_tn_WG, how = 'inner', on = ['Rv_ID', 'gene_name'])
df_tn_ALL.shape

(3990, 60)

## Do you want to order the columns in a special order? 

In [52]:
df_col_order = pd.read_excel(os.path.join(data_path,'column_order.xlsx'))
col_order = list(df_col_order.col_name.values)

In [53]:
df_tn_ALL = df_tn_ALL[col_order]

In [54]:
df_tn_ALL.shape

(3990, 60)

In [337]:
out_file = os.path.join(data_path,'Tn_library_DB.xlsx')
df_tn_ALL.to_excel(out_file, index = False)

In [338]:
df_tn_ALL.columns

Index(['Rv_ID', 'gene_name', '2003A_Sassetti', '2003B_Sassetti',
       '2005_Rengarajan', '2006_Joshi_GI_1', '2006_Joshi_GI_2',
       '2011_Griffin_2', '2012_Zhang', '2013_DeJesus', '2013_Zhang_1',
       '2013_Zhang_2', '2013_Zhang_3A', '2013_Zhang_3B', '2013_Zhang_3C',
       '2013_Zhang_3D', '2015_Mendum', '2015_Kieser_GI_1', '2015_Kieser_GI_2',
       '2015_Kieser_GI_3', '2016_Korte', '2016_Nambi', '2017A_DeJesus',
       '2017B_DeJesus_1A', '2017B_DeJesus_1B', '2017B_DeJesus_1C',
       '2017_Xu_1A', '2017_Xu_1B', '2017_Xu_1C', '2017_Xu_1D', '2017_Xu_1E',
       '2017_Mishra_1A', '2017_Mishra_1B', '2017_Mishra_1C', '2017_Mishra_1D',
       '2018_Carey_1A', '2018_Carey_1B', '2018_Carey_1C', '2018_Carey_1D',
       '2018_Carey_1E', '2018_Carey_1F', '2018_Carey_1G', '2018_Carey_1H',
       '2018_Rittershaus_1A', '2018_Rittershaus_1B', 'Rv3005c', 'Rv1565c',
       'ponA1', 'Rv3684', 'Rv3811', 'Rv0950', 'Rv3717', 'Rv3594', 'Rv3916c',
       'marP', 'Rv0954', 'Rv1432', 'Rv1096', 'Rv26

___________
___________
___________
___________


# (2003) Sassetti, Boyd, Rubin
Description: 
* 7H10 agar containing OADC for M. tuberculosis
* all genes in table_1.xls meet the criteria of: 
    * 1) having a ratio (insertion/genomic probe) < 0.2; 
    * 2) are reproducibly attenuated in M.tb and M. bolivs, with a T-test p_value < 0.05. 

# (2003) Sassetti and Rubin
Description: 
* in vivo (infection)

# (2006) Joshi, Sassetti
Description:
* This is a genetic interaction study with the "mce" loci (mce1 and mce4)
* Table_1 are positive genetic interactions with mce_1
* Table_2 are positive genetic interactions with mce_4

# (2011) Griffin and Sassetti
Description: 
* has two tables: 
    * A) for essentiality in minimal media w/ tylox, 0.2% Eth, 0.1% glycerol
    * B) specific to 0.01% cholesterol

# (2012) Zhang, Rubin
Description: 
* Grown in 7H10 media (as in Sassetti et al., 2001)

# (2013) Zhang, Rubin

Description: 
* Table 2: genes that were required for growth during infection
* Table 3: genes wherein mutations caused a growth defect in wild type mice but not in CD4-deficient mice. 
* Table(s) 4: series of in vitro stress conditions that model stresses thought to be present during in the immune competent host

# (2015) Kieser, Rubin
Description: 
* Genetic interactions with three different Peptidoglycan genes: 
* table_1A: PonA1; table_1B: PonA2; table_1C: LdtB; 

# (2018) Rittershaus, Sassetti

In [229]:
# preprocessing: 

In [262]:
file_list = ['2018_Rittershaus_Sassetti/table_2A.xlsx',
            '2018_Rittershaus_Sassetti/table_2B.xlsx']

In [237]:
for file in file_list:
    df_tn = pd.read_excel(os.path.join(dir_name, file))
    df_tn['Rv_ID']=['Rv'+rv.split('_')[-1] for rv in df_tn.Rv_ID.values]
    df_tn.to_excel(os.path.join(dir_name, file), index = False)

# (2017) Mishra and Sassetti

Preprocessing

In [358]:
file_list = ['2017_Mishra_Sassetti/table_1A.xlsx',
            '2017_Mishra_Sassetti/table_1B.xlsx',
            '2017_Mishra_Sassetti/table_1C.xlsx',
            '2017_Mishra_Sassetti/table_1D.xlsx']

In [359]:
for file in file_list:
    df_tn = pd.read_excel(os.path.join(dir_name, file))
    df_tn['Rv_ID']=['Rv'+rv.split('_')[-1] for rv in df_tn.Rv_ID.values]
    df_tn.to_excel(os.path.join(dir_name, file), index = False)

## (2013) Zhang and Rubin

In [205]:
file = '2013_Zhang_Rubin/table_2.xlsx'
df_tn = pd.read_excel(os.path.join(dir_name, file))
df_tn['Rv_ID']=[rv.strip() for rv in df_tn.Rv_ID.values]
df_tn.to_excel(os.path.join(dir_name, file), index = False)

### (2005) Rengarajan and Rubin

In [284]:
file = '2005_Rengarajan_Rubin/table_2.xls'
col_name = '2005_Rengarajan'
df_tn = pd.read_excel(os.path.join(dir_name, file))

In [286]:
df_tn_x = build_single_column_df(file, col_name)
# df_tn_WG = df_WG.merge(df_tn, how = 'left', on = 'Rv_ID')
df_tn_WG = merge_with_whole_genome(df_WG, col_name, df_tn_x)

In [287]:
ones = df_tn_WG[df_tn_WG['2005_Rengarajan'] == 1].Rv_ID.values

In [304]:
Rv_ID_old = df_tn[~df_tn.Rv_ID.isin(ones)].Rv_ID.values
Rv_ID_c = [rv+'c' for rv in Rv_ID_old]
Rv_ID_old[:5]

array(['Rv3596', 'Rv0503', 'Rv0107', 'Rv2214', 'Rv3556'], dtype=object)

In [305]:
rv_id_all = df_tn.Rv_ID.values
rv_id_all_c = []
for rv_id in rv_id_all:
    if rv_id in Rv_ID_old:
        rv_id_all_c.append(rv_id+'c')
    else:
        rv_id_all_c.append(rv_id)

In [308]:
df_tn['Rv_ID'] = rv_id_all_c
df_tn.to_excel(os.path.join(dir_name, file), index = False)